In [1]:
#@title
# Import necessary libraries
from os.path import basename, exists
from os import mkdir

def download(url,folder):
    filename = folder + basename(url)
    if not exists(folder):
        mkdir(folder)
    # fetches the file at the given url if it is not already present
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Notebooks/'
        + 'ModSimPy_Functions/modsim.py', 'ModSimPy_Functions/')

from ModSimPy_Functions.modsim import *
import pandas as pd
import numpy as np

Downloaded ModSimPy_Functions/modsim.py


In [4]:
rho = 1.2; g = 9.81                            
mass = 0.0025; diameter = 0.019 

v_term = 29.0
v_init = 0.0
height = 381 

params = dict(mass = mass, diameter=diameter, rho = rho,          
    g = g, v_init = v_init,  v_term = v_term,        
    height = height, t_end = 30)

In [6]:
def make_system(params):
    mass, diameter, rho, g, v_init, v_term, height, t_end = params.values()
    
    init = pd.Series(dict(y=height, v=v_init))

    area = 3.1415 * (diameter/2)**2
    C_d = (2 * mass * g / (rho * area * v_term**2))

    return dict(init=init,
                  area=area,
                  C_d=C_d,
                  mass=mass,
                  rho=rho,
                  g=g,
                  v_term = v_term,
                  t_end=t_end)

In [9]:
def slope_func(t, state, system):

    y, v = state
    rho, C_d, area = system['rho'], system['C_d'], system['area']
    mass, g = system['mass'], system['g']
    
    f_drag = rho * v**2 * C_d * area / 2
    
    if v < 0:
        a_drag = f_drag / mass 
    else:
        a_drag = -f_drag / mass
    
    dydt = v
    dvdt = -g + a_drag 
    
    return dydt, dvdt

def event_func(t, state, system):
    y, v = state
    return y

In [10]:
results, details = run_solve_ivp(system, slope_func, events=event_func)